In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import svhn
import numpy as np
import matplotlib.pyplot as plt

In [21]:
class Decoder_Block(nn.Module):
    def __init__ (self,num_layers=2,initial_filter=1024):
        super(Decoder_Block,self).__init__()
        self.relu = nn.ReLU()
        self.layers = []
        out_channel = initial_filter//2
        for i in range(num_layers):
            self.layers.append(nn.Conv2d(initial_filter,out_channel,kernel_size=3,padding=1))
            self.layers.append(nn.ReLU())
            initial_filter = out_channel
        self.block = nn.Sequential(*self.layers)

    def forward(self,x):
        x = self.block(x)
        print(x.shape)
        return x 

In [22]:
test_tensor = torch.randn(1, 1024, 28, 28)
model = Decoder_Block()
print(model)
output = model(test_tensor)
model

Decoder_Block(
  (relu): ReLU()
  (block): Sequential(
    (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
)
torch.Size([1, 512, 28, 28])


Decoder_Block(
  (relu): ReLU()
  (block): Sequential(
    (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
)

In [23]:
class Decoder(nn.Module):
    def __init__(self,num_blocks=5,in_channel=1024,final_channel=64):
        super(Decoder,self).__init__()
        self.blocks = []
        for i in reversed(range(1,num_blocks)):
            initial_filter = final_channel*(2**(i))
            self.blocks.append(Decoder_Block(initial_filter=initial_filter,num_layers=2))
            in_channel=initial_filter*2
            
        self.decoder = nn.Sequential(*self.blocks)
    def forward(self,x):
        x = self.decoder(x)


In [24]:
model = Decoder()
model(test_tensor)

torch.Size([1, 512, 28, 28])
torch.Size([1, 256, 28, 28])
torch.Size([1, 128, 28, 28])
torch.Size([1, 64, 28, 28])


In [28]:
x = nn.ConvTranspose2d(1024,512,2,stride=2)(test_tensor)
x.shape
torch.cat([x,test_tensor],dim=1)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 56 but got size 28 for tensor number 1 in the list.